In [ ]:
import pandas as pd
import torch
import pickle
import matplotlib.pyplot as plt

# 📥 Load CSV data
df = pd.read_csv("house_prices.csv")  # Replace with your actual path

# 🧹 Preprocess
X_np = df.drop(columns=["price"]).values
y_np = df["price"].values

# Convert to tensors
X = torch.tensor(X_np, dtype=torch.float32)
y = torch.tensor(y_np, dtype=torch.float32).view(-1, 1)

# 🔢 Initialize weights and bias manually
n_features = X.shape[1]
W = torch.randn(n_features, 1, requires_grad=True)
b = torch.randn(1, requires_grad=True)

# 🧮 Custom MSE loss function
def mse(pred, target):
    return torch.mean((pred - target) ** 2)

# 📊 Plotting function
def plot_fit(epoch, X, y, W, b):
    if X.shape[1] != 1:
        print(f"Skipping plot at epoch {epoch}: only works for 1 feature.")
        return

    with torch.no_grad():
        x_vals = X[:, 0].numpy()
        y_vals = y.numpy().flatten()
        y_pred = (X @ W + b).numpy().flatten()

        plt.figure(figsize=(8, 5))
        plt.scatter(x_vals, y_vals, label="Actual", alpha=0.6)
        plt.plot(x_vals, y_pred, color="red", label="Predicted Line")
        plt.title(f"Fitment at Epoch {epoch}")
        plt.xlabel("Feature")
        plt.ylabel("Price")
        plt.legend()
        plt.grid(True)
        plt.show()

# 🔁 Training loop
lr = 0.01
epochs = 10000

for epoch in range(epochs):
    y_pred = X @ W + b
    loss = mse(y_pred, y)

    loss.backward()  # Compute gradients of loss w.r.t. W and b
    # Gradient descent step (update weights and bias)
    with torch.no_grad():  # Temporarily disable gradient tracking
        W -= lr * W.grad  # Update weights using gradients and learning rate
        b -= lr * b.grad  # Update bias using gradients and learning rate
        W.grad.zero_()    # Reset gradients for W to zero for next iteration
        b.grad.zero_()    # Reset gradients for b to zero for next iteration

    if epoch % 1000 == 0:  # Every 1000 epochs, print progress and plot fit
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")  # Show current epoch and loss
        plot_fit(epoch, X, y, W, b)  # Visualize fit for 1D feature

# 💾 Save weights and bias to pickle
model_params = {"W": W.detach(), "b": b.detach()}  # Prepare model parameters for saving (remove gradients)
with open("manual_house_price_model.pkl", "wb") as f:  # Open file for writing in binary mode
    pickle.dump(model_params, f)  # Save parameters to file

print("✅ Model trained and saved to 'manual_house_price_model.pkl'")  # Confirm training